# 📊 UPI Transactions 2024 — Exploratory Data Analysis

This notebook explores the `upi_transactions_2024.csv` dataset to understand:
- Column structure and data types
- Transaction amount distributions
- Fraud patterns and peak hours
- Device, network, and regional breakdowns
- Age group and merchant category segmentation

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import warnings
warnings.filterwarnings('ignore')

# Load dataset
df = pd.read_csv('../ml/upi_transactions_2024.csv')
df.columns = [c.strip().lower().replace(' ', '_') for c in df.columns]

# Parse datetime if present
for col in ['timestamp', 'date', 'transaction_date', 'datetime']:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')
        df['hour_of_day'] = df[col].dt.hour
        df['day_of_week'] = df[col].dt.day_name()
        df['is_weekend'] = df[col].dt.dayofweek >= 5
        break

print(f'Shape: {df.shape}')
print(f'Columns: {df.columns.tolist()}')

In [ ]:
# ── 1. Dataset Overview ────────────────────────────────────────────────────────
print('=== DATA TYPES ===')
print(df.dtypes)
print('\n=== HEAD ===')
df.head(5)

In [ ]:
# ── 2. Missing Values ──────────────────────────────────────────────────────────
missing = df.isnull().sum()
print('Missing values per column:')
print(missing[missing > 0] if missing.any() else 'No missing values!')

In [ ]:
# ── 3. Transaction Amount Distribution ───────────────────────────────────────
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

axes[0].hist(df['amount'].dropna(), bins=60, color='#4f46e5', edgecolor='white', alpha=0.85)
axes[0].set_title('Transaction Amount Distribution', fontweight='bold')
axes[0].set_xlabel('Amount (₹)')
axes[0].set_ylabel('Frequency')
axes[0].xaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'₹{x:,.0f}'))

axes[1].hist(np.log1p(df['amount'].dropna()), bins=60, color='#7c3aed', edgecolor='white', alpha=0.85)
axes[1].set_title('Log-Transformed Amount Distribution', fontweight='bold')
axes[1].set_xlabel('log(Amount + 1)')

plt.tight_layout()
plt.show()

print(df['amount'].describe().apply(lambda x: f'₹{x:,.2f}'))

In [ ]:
# ── 4. Merchant Category Analysis ────────────────────────────────────────────
if 'merchant_category' in df.columns:
    cat_stats = df.groupby('merchant_category').agg(
        count=('amount', 'count'),
        avg_amount=('amount', 'mean'),
        fraud_rate=('fraud_flag', 'mean')
    ).round(2)
    cat_stats['fraud_rate_pct'] = (cat_stats['fraud_rate'] * 100).round(2)
    cat_stats = cat_stats.sort_values('count', ascending=False)

    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    cat_stats['count'].plot(kind='bar', ax=axes[0], color='#4f46e5', alpha=0.85)
    axes[0].set_title('Transaction Count by Category', fontweight='bold')
    axes[0].tick_params(axis='x', rotation=45)

    cat_stats['avg_amount'].plot(kind='bar', ax=axes[1], color='#0ea5e9', alpha=0.85)
    axes[1].set_title('Avg Amount by Category (₹)', fontweight='bold')
    axes[1].tick_params(axis='x', rotation=45)

    cat_stats['fraud_rate_pct'].plot(kind='bar', ax=axes[2], color='#ef4444', alpha=0.85)
    axes[2].set_title('Fraud Rate (%) by Category', fontweight='bold')
    axes[2].tick_params(axis='x', rotation=45)

    plt.tight_layout()
    plt.show()
    print(cat_stats)

In [ ]:
# ── 5. Peak Hours Analysis ───────────────────────────────────────────────────
if 'hour_of_day' in df.columns:
    hourly = df.groupby('hour_of_day').agg(
        count=('amount', 'count'),
        avg_amount=('amount', 'mean'),
        fraud_rate=('fraud_flag', 'mean')
    ).round(2)

    fig, axes = plt.subplots(1, 2, figsize=(14, 4))
    axes[0].bar(hourly.index, hourly['count'], color='#4f46e5', alpha=0.85)
    axes[0].set_title('Transactions by Hour of Day', fontweight='bold')
    axes[0].set_xlabel('Hour')
    axes[0].set_xticks(range(0, 24))

    axes[1].plot(hourly.index, hourly['fraud_rate'] * 100, color='#ef4444', marker='o', linewidth=2)
    axes[1].set_title('Fraud Rate (%) by Hour', fontweight='bold')
    axes[1].set_xlabel('Hour')
    axes[1].set_ylabel('Fraud Rate (%)')
    axes[1].set_xticks(range(0, 24))

    plt.tight_layout()
    plt.show()

    peak_hour = int(hourly['count'].idxmax())
    print(f'Peak transaction hour: {peak_hour:02d}:00–{peak_hour+1:02d}:00 ({hourly["count"].max():,} transactions)')

In [ ]:
# ── 6. Device & Network Comparison ──────────────────────────────────────────
for col in ['device_type', 'network_type']:
    if col in df.columns:
        grp = df.groupby(col).agg(
            count=('amount', 'count'),
            avg_amount=('amount', 'mean'),
            fraud_rate=('fraud_flag', 'mean')
        ).round(2)
        grp['fraud_rate_pct'] = (grp['fraud_rate'] * 100).round(2)
        print(f'\n=== {col.replace("_", " ").title()} ===')
        print(grp[['count', 'avg_amount', 'fraud_rate_pct']])

In [ ]:
# ── 7. Age Group Segmentation ────────────────────────────────────────────────
if 'age_group' in df.columns:
    age_stats = df.groupby('age_group').agg(
        count=('amount', 'count'),
        avg_amount=('amount', 'mean'),
        fraud_rate=('fraud_flag', 'mean')
    ).round(2)
    age_stats['fraud_rate_pct'] = (age_stats['fraud_rate'] * 100).round(2)
    print('Age Group Segmentation:')
    print(age_stats[['count', 'avg_amount', 'fraud_rate_pct']])

    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    age_stats['avg_amount'].plot(kind='bar', ax=axes[0], color='#0ea5e9', alpha=0.85)
    axes[0].set_title('Avg Amount by Age Group (₹)', fontweight='bold')
    axes[0].tick_params(axis='x', rotation=30)

    age_stats['fraud_rate_pct'].plot(kind='bar', ax=axes[1], color='#ef4444', alpha=0.85)
    axes[1].set_title('Fraud Rate (%) by Age Group', fontweight='bold')
    axes[1].tick_params(axis='x', rotation=30)
    plt.tight_layout()
    plt.show()

In [ ]:
# ── 8. State-Level Analysis ──────────────────────────────────────────────────
if 'state' in df.columns:
    state_stats = df.groupby('state').agg(
        count=('amount', 'count'),
        avg_amount=('amount', 'mean'),
        fraud_rate=('fraud_flag', 'mean')
    ).round(2)
    state_stats['fraud_rate_pct'] = (state_stats['fraud_rate'] * 100).round(2)
    top10 = state_stats.sort_values('count', ascending=False).head(10)
    print('Top 10 States by Transaction Volume:')
    print(top10[['count', 'avg_amount', 'fraud_rate_pct']])

In [ ]:
# ── 9. Overall Risk Summary ──────────────────────────────────────────────────
total = len(df)
fraud_count = int(df['fraud_flag'].sum()) if 'fraud_flag' in df.columns else 0
fraud_rate = round(fraud_count / total * 100, 2)

failed_count = 0
failure_rate = 0
if 'status' in df.columns:
    failed_count = int((df['status'].str.upper() == 'FAILED').sum())
    failure_rate = round(failed_count / total * 100, 2)

print('=== RISK SUMMARY ===')
print(f'Total Transactions : {total:,}')
print(f'Fraud Transactions : {fraud_count:,} ({fraud_rate}%)')
print(f'Failed Transactions: {failed_count:,} ({failure_rate}%)')
print(f'Avg Amount         : ₹{df["amount"].mean():,.2f}')
print(f'Merchant Categories: {df["merchant_category"].nunique() if "merchant_category" in df.columns else "N/A"}')
print(f'States Covered     : {df["state"].nunique() if "state" in df.columns else "N/A"}')